![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/26.Chunk_Mapping.ipynb)

# Chunk Mapping

## Colab Setup

In [ ]:
import json
import os

from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)

# Adding license key-value pairs to environment variables
os.environ.update(license_keys)

In [2]:
%%capture

# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

In [3]:
import json
import os
from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl
import sparknlp
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

params = {"spark.driver.memory":"16G", 
          "spark.kryoserializer.buffer.max":"2000M", 
          "spark.driver.maxResultSize":"2000M"} 

print ("Spark NLP Version :", sparknlp.version())
print ("Spark NLP_JSL Version :", sparknlp_jsl.version())


spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

spark

Spark NLP Version : 3.4.2
Spark NLP_JSL Version : 3.5.0


# 1- Pretrained Chunk Mapper Models

## 1.1- Drug Action Treatment Mapper

Pretrained `drug_action_treatment_mapper` model maps drugs with their corresponding `action` and `treatment` through `ChunkMapperModel()` annotator. <br/>


**Action** of drug refers to the function of a drug in various body systems. <br/>
**Treatment** refers to which disease the drug is used to treat. 

We can choose which option we want to use by setting `setRel()` parameter of `ChunkMapperModel()`
 

We will create a pipeline consisting `bert_token_classifier_drug_development_trials` ner model to extract ner chunk as well as `ChunkMapperModel()`. <br/>
 Also, we will set the `.setRel()` parameter with `action` and see the results. 

In [44]:
#ChunkMapper Pipeline
document_assembler = DocumentAssembler()\
      .setInputCol('text')\
      .setOutputCol('document')

sentence_detector = SentenceDetector()\
      .setInputCols(["document"])\
      .setOutputCol("sentence")

tokenizer = Tokenizer()\
      .setInputCols("sentence")\
      .setOutputCol("token")

ner =  MedicalBertForTokenClassifier.pretrained("bert_token_classifier_drug_development_trials", "en", "clinical/models")\
      .setInputCols("token","sentence")\
      .setOutputCol("ner")

nerconverter = NerConverter()\
      .setInputCols("sentence", "token", "ner")\
      .setOutputCol("ner_chunk")

#drug_action_treatment_mapper with "action" mappings
chunkerMapper= ChunkMapperModel().pretrained("drug_action_treatment_mapper", "en", "clinical/models")\
    .setInputCols(["ner_chunk"])\
    .setOutputCol("action_mappings")\
    .setRel("action")


pipeline = Pipeline().setStages([document_assembler,
                                 sentence_detector,
                                 tokenizer,
                                 ner, 
                                 nerconverter, 
                                 chunkerMapper])

text = [
    ["""The patient was female and patient of Dr. X. and she was given Dermovate, Aspagin"""]
]


test_data = spark.createDataFrame(text).toDF("text")

res = pipeline.fit(test_data).transform(test_data)


bert_token_classifier_drug_development_trials download started this may take some time.
Approximate size to download 382.4 MB
[OK!]
drug_action_treatment_mapper download started this may take some time.
Approximate size to download 8.3 MB
[OK!]


Chunks detected by ner model

In [45]:
res.select(F.explode('ner_chunk.result').alias("chunks")).show(truncate=False)

+---------+
|chunks   |
+---------+
|Dermovate|
|Aspagin  |
+---------+



Checking mapping results

In [46]:
res.select("action_mappings.result").show(truncate=False)

+------------------------------+
|result                        |
+------------------------------+
|[Anti-Inflammatory, Analgesic]|
+------------------------------+



In [47]:
res.selectExpr("action_mappings.metadata").show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|metadata                                                                                                                                                                                                                                                                                                                       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[{chunk -> 0, relation -> action,

As you see above under the ***metadata*** column, if exist, we can see all the relations for each chunk. <br/>


In [48]:
res.select(F.explode(F.arrays_zip("ner_chunk.result", "action_mappings.result", "action_mappings.metadata")).alias("col"))\
    .select(F.expr("col['0']").alias("ner_chunk"),
            F.expr("col['1']").alias("mapping_result"),
            F.expr("col['2']['all_relations']").alias("all_relations")).show(truncate=False)

+---------+-----------------+------------------------------------------------------------+
|ner_chunk|mapping_result   |all_relations                                               |
+---------+-----------------+------------------------------------------------------------+
|Dermovate|Anti-Inflammatory|Corticosteroids::: Dermatological Preparations:::Very Strong|
|Aspagin  |Analgesic        |Anti-Inflammatory:::Antipyretic                             |
+---------+-----------------+------------------------------------------------------------+



Now, let's set the `.setRel("treatment")` and see the results. 

In [49]:
#drug_action_treatment_mapper with "treatment" mappings
chunkerMapper= ChunkMapperModel().pretrained("drug_action_treatment_mapper", "en", "clinical/models")\
    .setInputCols(["ner_chunk"])\
    .setOutputCol("action_mappings")\
    .setRel("treatment")

pipeline = Pipeline().setStages([document_assembler,
                                 sentence_detector,
                                 tokenizer,
                                 ner, 
                                 nerconverter, 
                                 chunkerMapper])

text = [
    ["""The patient was female and patient of Dr. X. and she was given Dermovate, Aspagin"""]
]

test_data = spark.createDataFrame(text).toDF("text")

res = pipeline.fit(test_data).transform(test_data)


drug_action_treatment_mapper download started this may take some time.
Approximate size to download 8.3 MB
[OK!]


In [50]:
res.select(F.explode('ner_chunk.result').alias("chunks")).show(truncate=False)

+---------+
|chunks   |
+---------+
|Dermovate|
|Aspagin  |
+---------+



In [51]:
res.selectExpr("action_mappings.metadata").show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|metadata                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

Here are the ***treatment*** mappings and all relations under the metadata column. 

In [52]:
res.select(F.explode(F.arrays_zip("ner_chunk.result", "action_mappings.result", "action_mappings.metadata")).alias("col"))\
    .select(F.expr("col['0']").alias("ner_chunk"),
            F.expr("col['1']").alias("mapping_result"),
            F.expr("col['2']['all_relations']").alias("all_relations")).show(truncate=False)

+---------+----------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ner_chunk|mapping_result        |all_relations                                                                                                                                                                                                          |
+---------+----------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Dermovate|Lupus                 |Discoid Lupus Erythematosus:::Empeines:::Psoriasis:::Eczema                                                                                                                                                          

## 1.2- Section Header Normalizer Mapper

We have `normalized_section_header_mapper` model that normalizes the section headers in clinical notes. It returns two levels of normalization called `level_1` and `level_2`. <br/>

**level_1** refers to the most comprehensive "section header" for the corresponding chunk while **level_2** refers to the second comprehensive one.

Let's create a piepline with `normalized_section_header_mapper` and see how it works

In [ ]:
document_assembler = DocumentAssembler()\
       .setInputCol('text')\
       .setOutputCol('document')

sentence_detector = SentenceDetector()\
       .setInputCols(["document"])\
       .setOutputCol("sentence")

tokenizer = Tokenizer()\
      .setInputCols("sentence")\
      .setOutputCol("token")

embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en","clinical/models")\
      .setInputCols(["sentence", "token"])\
      .setOutputCol("word_embeddings")

clinical_ner = MedicalNerModel.pretrained("ner_jsl_slim", "en", "clinical/models")\
      .setInputCols(["sentence","token", "word_embeddings"])\
      .setOutputCol("ner")

ner_converter = NerConverter()\
      .setInputCols(["sentence", "token", "ner"])\
      .setOutputCol("ner_chunk")\
      .setWhiteList(["Header"])

chunkerMapper = ChunkMapperModel.pretrained("normalized_section_header_mapper", "en", "clinical/models") \
       .setInputCols("ner_chunk")\
       .setOutputCol("mappings")\
       .setRel("level_1") #or level_2

pipeline = Pipeline().setStages([document_assembler,
                                sentence_detector,
                                tokenizer, 
                                embeddings,
                                clinical_ner, 
                                ner_converter, 
                                chunkerMapper])

sentences = [
    ["""ADMISSION DIAGNOSIS Right pleural effusion and suspected malignant mesothelioma.
        PRINCIPAL DIAGNOSIS Right pleural effusion, suspected malignant mesothelioma.
        GENERAL REVIEW Right pleural effusion, firm nodules, diffuse scattered throughout the right pleura and diaphragmatic surface.
    """]]

test_data = spark.createDataFrame(sentences).toDF("text")
res = pipeline.fit(test_data).transform(test_data)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_jsl_slim download started this may take some time.
Approximate size to download 14.4 MB
[OK!]
normalized_section_header_mapper download started this may take some time.
Approximate size to download 13.9 KB
[OK!]


Checking the headers detected by ner model

In [ ]:
res.select(F.explode('ner_chunk.result').alias("chunks")).show(truncate=False)

+-------------------+
|chunks             |
+-------------------+
|ADMISSION DIAGNOSIS|
|PRINCIPAL DIAGNOSIS|
|GENERAL REVIEW     |
+-------------------+



Checking mapping results

In [ ]:
res.select("mappings.result").show(truncate=False)

+-----------------------------------+
|result                             |
+-----------------------------------+
|[DIAGNOSIS, DIAGNOSIS, REVIEW TYPE]|
+-----------------------------------+



In [ ]:
res.select(F.explode(F.arrays_zip("ner_chunk.result", "mappings.result")).alias("col"))\
    .select(F.expr("col['0']").alias("ner_chunk"),
            F.expr("col['1']").alias("mapping_result")).show(truncate=False)

+-------------------+--------------+
|ner_chunk          |mapping_result|
+-------------------+--------------+
|ADMISSION DIAGNOSIS|DIAGNOSIS     |
|PRINCIPAL DIAGNOSIS|DIAGNOSIS     |
|GENERAL REVIEW     |REVIEW TYPE   |
+-------------------+--------------+



As you see above, we can see the "level_1" based normalized version of each section header.

# 2- Creating a Mapper Model

There is `ChunkMapperApproach()` to create your own mapper model. <br/>

This receives an `ner_chunk` and a Json with a mapping of ner entities and relations, and returns the `ner_chunk` augmented with the relations from the Json ontology. <br/> We give the path of json file to the `setDictionary()` parameter.




Let's create an example Json with single key, then create a drug mapper model. This model will match the given drug name (only "metformin" for our example) with correpsonding action and treatment.  

The format of json file should be like following:

````
{
  "mappings": [
    {
      "key": "metformin",
      "relations": [
        {
          "key": "action",
          "values" : ["hypoglycemic", "Drugs Used In Diabets"]
        },
        {
          "key": "treatment",
          "values" : ["diabetes", "t2dm"]
        }
      ]
    }
  ]
}


By using `setRel()` parameter, we tell the model which type of mapping we want. In our case, if we want from our model to return **action** mapping, we set the parameter as `setRel("action")`,  we set as `setRel("treatment")` for **treatment**

Let's create a pipeline and see it in action. 

In [ ]:
document_assembler = DocumentAssembler()\
      .setInputCol('text')\
      .setOutputCol('document')

sentence_detector = SentenceDetector()\
      .setInputCols(["document"])\
      .setOutputCol("sentence")

tokenizer = Tokenizer()\
      .setInputCols("sentence")\
      .setOutputCol("token")

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
      .setInputCols(["sentence", "token"])\
      .setOutputCol("embeddings")

#NER model to detect drug in the text
clinical_ner = MedicalNerModel.pretrained("ner_posology_small","en","clinical/models")\
	    .setInputCols(["sentence","token","embeddings"])\
	    .setOutputCol("ner")\
      .setLabelCasing("upper")
 
ner_converter = NerConverter()\
      .setInputCols(["sentence", "token", "ner"])\
      .setOutputCol("ner_chunk")\
      .setWhiteList(["DRUG"])

chunkerMapper = ChunkMapperApproach()\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("mappings")\
      .setDictionary("/content/sample_drug.json")\
      .setRel("action") #or treatment

pipeline = Pipeline().setStages([document_assembler,
                                 sentence_detector,
                                 tokenizer, 
                                 word_embeddings,
                                 clinical_ner, 
                                 ner_converter, 
                                 chunkerMapper])

text = ["The patient was given 1 unit of metformin daily."]

test_data = spark.createDataFrame([text]).toDF("text")

model = pipeline.fit(test_data)
res= model.transform(test_data)


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_posology_small download started this may take some time.
Approximate size to download 13.9 MB
[OK!]


In [ ]:
res.printSchema()

root
 |-- text: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- sentence: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true

Checking the ner result

In [ ]:
res.select(F.explode('ner_chunk.result').alias("chunks")).show(truncate=False)

+---------+
|chunks   |
+---------+
|metformin|
+---------+



Checking the mapper result

In [ ]:
res.selectExpr("mappings.metadata").show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------+
|metadata                                                                                                                            |
+------------------------------------------------------------------------------------------------------------------------------------+
|[{chunk -> 0, relation -> action, confidence -> 0.9906, all_relations -> Drugs Used In Diabets, entity -> metformin, sentence -> 0}]|
+------------------------------------------------------------------------------------------------------------------------------------+



In [ ]:
res.select(F.explode(F.arrays_zip("ner_chunk.result", "mappings.result", "mappings.metadata")).alias("col"))\
    .select(F.expr("col['0']").alias("ner_chunk"),
            F.expr("col['1']").alias("mapping_result"),
            F.expr("col['2']['all_relations']").alias("all_relations")).show(truncate=False)

+---------+--------------+---------------------+
|ner_chunk|mapping_result|all_relations        |
+---------+--------------+---------------------+
|metformin|hypoglycemic  |Drugs Used In Diabets|
+---------+--------------+---------------------+



As you see, the model that we created with `ChunkMapperApproach()` succesfully mapped "metformin". Under the metadata, we can see all relations that we defined in Json. 

### 2.1- Save the model to disk 

Now, we will save our model and use it with `ChunkMapperModel()`

In [ ]:
model.stages[-1].write().save("models/drug_mapper")

Using the saved model. This time we will check 'treatment' mappings results


In [ ]:
document_assembler = DocumentAssembler()\
      .setInputCol('text')\
      .setOutputCol('document')

sentence_detector = SentenceDetector()\
      .setInputCols(["document"])\
      .setOutputCol("sentence")

tokenizer = Tokenizer()\
      .setInputCols("sentence")\
      .setOutputCol("token")

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
      .setInputCols(["sentence", "token"])\
      .setOutputCol("embeddings")

#NER model to detect drug in the text
clinical_ner = MedicalNerModel.pretrained("ner_posology_small","en","clinical/models")\
	    .setInputCols(["sentence","token","embeddings"])\
	    .setOutputCol("ner")\
      .setLabelCasing("upper")
 
ner_converter = NerConverter()\
      .setInputCols(["sentence", "token", "ner"])\
      .setOutputCol("ner_chunk")\
      .setWhiteList(["DRUG"])

chunkerMapper = ChunkMapperModel.load("/content/models/drug_mapper")\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("mappings")\
      .setRel("treatment") 

pipeline = Pipeline().setStages([document_assembler,
                                 sentence_detector,
                                 tokenizer, 
                                 word_embeddings,
                                 clinical_ner, 
                                 ner_converter, 
                                 chunkerMapper])

text = ["The patient was given 1 unit of metformin daily."]

test_data = spark.createDataFrame([text]).toDF("text")

model = pipeline.fit(test_data)
res= model.transform(test_data)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_posology_small download started this may take some time.
Approximate size to download 13.9 MB
[OK!]


In [ ]:
res.selectExpr("mappings.metadata").show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------+
|metadata                                                                                                              |
+----------------------------------------------------------------------------------------------------------------------+
|[{chunk -> 0, relation -> treatment, confidence -> 0.9906, all_relations -> t2dm, entity -> metformin, sentence -> 0}]|
+----------------------------------------------------------------------------------------------------------------------+



In [ ]:
res.select(F.explode(F.arrays_zip("ner_chunk.result", "mappings.result", "mappings.metadata")).alias("col"))\
    .select(F.expr("col['0']").alias("ner_chunk"),
            F.expr("col['1']").alias("mapping_result"),
            F.expr("col['2']['all_relations']").alias("all_relations")).show(truncate=False)

+---------+--------------+-------------+
|ner_chunk|mapping_result|all_relations|
+---------+--------------+-------------+
|metformin|diabetes      |t2dm         |
+---------+--------------+-------------+



As you see above, we created our own drug mapper model successfully. 

# 3- Section Header Normalizer Mapper with ChunkSentenceSplitter

`ChunkSentenceSplitter()` annotator splits documents or sentences by chunks provided. <br/> For detailed usage of this annotator, visit [this notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/18.Chunk_Sentence_Splitter.ipynb) <br/>

In this section, we will do the following steps; 
- Detect "section headers" in given text through Ner model
- Split the given text by headers with `ChunkSentenceSplitter()`
- Normalize the `ChunkSentenceSplitter()` outputs with `normalized_section_header_mapper` model. 

Let's start with creating Ner pipeline to detect "Header" 

In [20]:
sentences = [
    ["""ADMISSION DIAGNOSIS Right pleural effusion and suspected malignant mesothelioma.
        PRINCIPAL DIAGNOSIS Right pleural effusion, suspected malignant mesothelioma.
        REVIEW OF SYSTEMS Right pleural effusion, firm nodules, diffuse scattered throughout the right pleura and diaphragmatic surface.
    """]]

df= spark.createDataFrame(sentences).toDF("text")

In [21]:
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

tokenizer= Tokenizer()\
      .setInputCols(["document"])\
      .setOutputCol("token")

tokenClassifier = MedicalBertForTokenClassifier.pretrained("bert_token_classifier_ner_jsl_slim", "en", "clinical/models")\
    .setInputCols("token", "document")\
    .setOutputCol("ner")\
    .setCaseSensitive(True)

ner_converter = NerConverter() \
      .setInputCols(["document", "token", "ner"]) \
      .setOutputCol("ner_chunk")\
      .setWhiteList(["Header"])

pipeline = Pipeline(
    stages = [
        documentAssembler,
        tokenizer,
        tokenClassifier,
        ner_converter
    ])
 
empty_df = spark.createDataFrame([[""]]).toDF('text')
pipeline_model = pipeline.fit(empty_df)

bert_token_classifier_ner_jsl_slim download started this may take some time.
Approximate size to download 385.7 MB
[OK!]


In [22]:
result = pipeline_model.transform(df)
result.selectExpr('explode(ner_chunk)').show(truncate=False)

+------------------------------------------------------------------------------------------------------------------+
|col                                                                                                               |
+------------------------------------------------------------------------------------------------------------------+
|{chunk, 0, 18, ADMISSION DIAGNOSIS, {entity -> Header, sentence -> 0, chunk -> 0, confidence -> 0.9994346}, []}   |
|{chunk, 89, 107, PRINCIPAL DIAGNOSIS, {entity -> Header, sentence -> 0, chunk -> 1, confidence -> 0.99020165}, []}|
|{chunk, 175, 191, REVIEW OF SYSTEMS, {entity -> Header, sentence -> 0, chunk -> 2, confidence -> 0.9989373}, []}  |
+------------------------------------------------------------------------------------------------------------------+



Now, we have our header entities. We will split the text by the headers.

In [23]:
#applying ChunkSentenceSplitter 
chunkSentenceSplitter = ChunkSentenceSplitter()\
    .setInputCols("document","ner_chunk")\
    .setOutputCol("paragraphs")\
    .setGroupBySentences(False)

paragraphs = chunkSentenceSplitter.transform(result)

In [24]:
paragraphs.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|               token|                 ner|           ner_chunk|          paragraphs|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|ADMISSION DIAGNOS...|[{document, 0, 30...|[{token, 0, 8, AD...|[{named_entity, 0...|[{chunk, 0, 18, A...|[{document, 0, 89...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+



In [25]:
pd.set_option('display.max_colwidth', None)
result_df = paragraphs.selectExpr("explode(paragraphs) as result").selectExpr("result.result","result.metadata.entity", "result.metadata.splitter_chunk").toPandas()
result_df.head()

,result,entity,splitter_chunk
0,ADMISSION DIAGNOSIS Right pleural effusion and suspected malignant mesothelioma.\n,Header,ADMISSION DIAGNOSIS
1,"PRINCIPAL DIAGNOSIS Right pleural effusion, suspected malignant mesothelioma.\n",Header,PRINCIPAL DIAGNOSIS
2,"REVIEW OF SYSTEMS Right pleural effusion, firm nodules, diffuse scattered throughout the right pleura and diaphragmatic surface.\n",Header,REVIEW OF SYSTEMS


As you see, we have our splitted text and **section headers**. <br/>
Now we will normalize this section headers with `normalized_section_header_mapper`

In [26]:
chunkerMapper = ChunkMapperModel.pretrained("normalized_section_header_mapper", "en", "clinical/models") \
       .setInputCols("ner_chunk")\
       .setOutputCol("mappings")\
       .setRel("level_1") #or level_2

normalized_df= chunkerMapper.transform(paragraphs)

normalized_section_header_mapper download started this may take some time.
Approximate size to download 13.9 KB
[OK!]


In [27]:
normalized_df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|               token|                 ner|           ner_chunk|          paragraphs|            mappings|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|ADMISSION DIAGNOS...|[{document, 0, 30...|[{token, 0, 8, AD...|[{named_entity, 0...|[{chunk, 0, 18, A...|[{document, 0, 89...|[{labeled_depende...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+



In [28]:
normalized_df= normalized_df.select(F.explode(F.arrays_zip("ner_chunk.result", "mappings.result")).alias("col"))\
                            .select(F.expr("col['0']").alias("ner_chunk"),
                                    F.expr("col['1']").alias("normalized_headers")).toPandas()
normalized_df.head()

,ner_chunk,normalized_headers
0,ADMISSION DIAGNOSIS,DIAGNOSIS
1,PRINCIPAL DIAGNOSIS,DIAGNOSIS
2,REVIEW OF SYSTEMS,REVIEW TYPE


Now, we have our normalized headers. We will merge it with `ChunkSentenceSplitter()` output

In [29]:
normalized_df= normalized_df.rename(columns={"ner_chunk": "splitter_chunk"})
df= pd.merge(result_df, normalized_df, on=["splitter_chunk"])

In [30]:
df.head()

,result,entity,splitter_chunk,normalized_headers
0,ADMISSION DIAGNOSIS Right pleural effusion and suspected malignant mesothelioma.\n,Header,ADMISSION DIAGNOSIS,DIAGNOSIS
1,"PRINCIPAL DIAGNOSIS Right pleural effusion, suspected malignant mesothelioma.\n",Header,PRINCIPAL DIAGNOSIS,DIAGNOSIS
2,"REVIEW OF SYSTEMS Right pleural effusion, firm nodules, diffuse scattered throughout the right pleura and diaphragmatic surface.\n",Header,REVIEW OF SYSTEMS,REVIEW TYPE


Ultimately, we have splitted paragraphs, headers and normalized headers. 